In [ ]:
import numpy as np
from sklearn.base import clone
from sklearn.model_selection import cross_val_predict, KFold
import catboost
from scipy.special import expit

In [ ]:
DATA_PATH="/mnt/public/MuID.npz"

In [ ]:
data = np.load(DATA_PATH)

In [ ]:
X_train = data["X_train"]
y_train = data["y_train"]
sWeight_train = data["sw_train"]
X_test = data["X_test"]
y_test = data["y_test"]
sWeight_test = data["sw_test"]

In [ ]:
CPU_COUNT = 10
CV = None
ITERATIONS = 1000

In [ ]:
params_regression = {
         "iterations": ITERATIONS,
         "verbose": False,
         "loss_function": "ConstrainedRegression",
         "thread_count": CPU_COUNT // CV if CV is not None else CPU_COUNT,
         #"learning_rate": 5e-2,
         #"l2_leaf_reg": 3,
         #"max_depth": 6,
         "leaf_estimation_method": "Gradient"}


params_likelihood = {
         "iterations": ITERATIONS,
         "verbose": False,
         "loss_function": "HonestLikelihood",
         "thread_count": CPU_COUNT // CV if CV is not None else CPU_COUNT,
         #"learning_rate": 5e-2,
         #"l2_leaf_reg": 3,
         #"max_depth": 6,
         "leaf_estimation_method": "Gradient"}

In [ ]:
#eval_set = catboost.Pool(data=X_test, label=y_test, weight=sWeight_test)
#eval_weights = catboost.Pool(data=X_test, label=sWeight_test)

In [ ]:
import catboost

In [ ]:
def transform_weights(x, weights, model, cv=None):
    model = clone(model)
    if cv is not None:
        if isinstance(cv, int):
            n_jobs=cv
            cv=KFold(cv, shuffle=True)
        else:
            n_jobs=cv.get_n_splits()
        res = cross_val_predict(model, x, weights, cv=cv, n_jobs=n_jobs)
    else:
        res = model.fit(x, weights).predict(x)
    if model.get_param("loss_function") == "ConstrainedRegression":
        return expit(res)
    else:
        return res

In [ ]:
params_classification = {
         "iterations": ITERATIONS,
         "verbose": False,
         "loss_function": "CrossEntropy",
         "thread_count": CPU_COUNT,
         #"learning_rate": 5e-2,
         #"l2_leaf_reg": 3,
         #"max_depth": 6,
         "leaf_estimation_method": "Gradient"}

In [ ]:
def roc_auc(classes : np.ndarray,
            predictions : np.ndarray,
            weights : np.ndarray = None) -> float:
    if weights is None:
        weights = np.ones_like(predictions)

    assert len(classes) == len(predictions) == len(weights)
    assert classes.ndim == predictions.ndim == weights.ndim == 1

    idx = np.argsort(predictions)

    predictions = predictions[idx]
    weights     = weights    [idx]
    classes     = classes    [idx]

    weights_0 = weights * (classes == 0)
    weights_1 = weights * (classes == 1)

    cumsum_0 = weights_0.cumsum()
    return (cumsum_0 * weights_1).sum() / (weights_1 * cumsum_0[-1]).sum()

In [ ]:
def scale(a):
    b = a - a.min()
    b /= b.max()
    return b

In [ ]:
def probabilities_from_s_weights(sw1, sw2):
    if sw1.ndim == 1:
        sw1 = np.vstack([sw1, 1 - sw1]).T

    if sw2.ndim == 1:
        sw2 = np.vstack([sw2, 1 - sw2]).T

    probs = np.dot(sw2, np.transpose(np.linalg.inv(np.dot(sw1.T, sw1)))) * sw1.sum(axis=0)
    return probs[:, 1]
pm_train = np.empty(len(X_train))
for particle_type in range(2):
    pid_selection = (y_train == particle_type)
    pm_train[pid_selection] = scale(probabilities_from_s_weights(sWeight_train[pid_selection],
                                                           sWeight_train[pid_selection]))

In [ ]:
ss = np.array([1e-4, 1e-3, 1e-2, 5e-2, 1e-1, 1])

In [ ]:
def get_scores(seed):
    np.random.seed(seed)
    res = []
    for sample_size in ss:
        res.append([])
        selection = np.random.choice(len(y_train), replace=False, size=int(sample_size*len(y_test)))
        this_X_train = X_train[selection]
        this_y_train = y_train[selection]
        this_sWeight_train = sWeight_train[selection]
        this_pm_train = pm_train[selection]
        model_noWeight = catboost.CatBoostClassifier(**params_classification).fit(this_X_train, this_y_train,
                                                                                  plot=True)
        res[-1].append(model_noWeight)
        model_naive = catboost.CatBoostClassifier(**params_classification).fit(
            this_X_train, this_y_train, sample_weight=this_sWeight_train, plot=True)
        res[-1].append(model_naive)
        for col, params in zip((this_sWeight_train, ), (params_regression, )):
            transformed_sweight = np.empty(len(this_X_train))
            for particle_type in range(2):
                pid_selection = (this_y_train == particle_type)
                transformed_sweight[pid_selection] = transform_weights(
                    this_X_train[pid_selection], col[pid_selection], catboost.CatBoostRegressor(**params),
                    cv=CV)
            model_smart = catboost.CatBoostClassifier(**params_classification).fit(
                this_X_train, this_y_train, sample_weight=transformed_sweight, plot=True)
            res[-1].append(model_smart)
        for model in res[-1]:
            print(roc_auc(y_test, model.predict_proba(X_test)[:, 1], sWeight_test), model.tree_count_)
    return res

In [ ]:
all_models = [get_scores(i) for i in range(10)]

In [ ]:
def bootstrap_roc_auc(labels, p, weights):
    scores = []
    for i in range(10):
        selection = np.random.choice(len(labels), replace=True, size=len(labels))
        scores.append(roc_auc(labels[selection], p[selection], weights[selection]))
    return np.array(scores)

In [ ]:
all_scores = [[[bootstrap_roc_auc(y_test,
                        model.predict(X_test, prediction_type="RawFormulaVal"),
                        sWeight_test)for model in x] for x in y] for y in all_models]

In [ ]:
all_scores_np = np.array(all_scores)

In [ ]:
mean_scores = all_scores_np.mean(axis=(0, 3))

In [ ]:
std_scores = all_scores_np.std(axis=(0, 3))

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
all_train_sizes = ss*len(X_train)

In [ ]:
fig, ax = plt.subplots(figsize=(3, 3))
ax.errorbar(all_train_sizes, mean_scores[:, 0], yerr=std_scores[:, 0], label="Ignoring weight")
ax.errorbar(all_train_sizes, mean_scores[:, 1],  yerr=std_scores[:, 1], label="sWeight")
ax.errorbar(all_train_sizes, mean_scores[:, 2],  yerr=std_scores[:, 2], label="Constrained MSE")
ax.legend()
ax.set_xscale("log")
ax.set_xlabel("Train size")
ax.set_ylabel("ROC AUC $\mu$ vs $\pi$")
plt.tight_layout()
fig.savefig("MuID_by_train_size.pdf", bbox="tight")